# Effects of migration

The genomic transmission graph can be broken down into many local subpopulations that together make up a metapopulation.  Here we examine a local subpopulation that is embedded within a much larger metapopulation.  

$m$ is the probability that a host within the local subpopulation acquired their infection from the metapopulation, and the number of such hosts per generation is $N_m$ = $mN_h$. These could be immigrants from the metapopulation or local residents who have been travelling outside the local area. 

We must modify our Markov chain simulation of coalescence times to account for migration.  Within `coalestr` this is split this into two components, with one [transmission probability matrix for the subpopulation](https://d-kwiat.github.io/gtg/subpop-matrix.html) and another [transition probability matrix for the metapopulation](https://d-kwiat.github.io/gtg/transition-probability.html).

In `coalestr` we can examine the effects of migration within a hierarchical population structure as follows:

1. We create a metapopulation and obtain its probability distribution of coalescence times and heterozygosity estimates.
 * specify a history of transmission parameters for the metapopulation
 * pass this to `coalestr.Population()` and give the metapopulation a name
 * run `get_coalescent()` for the metapopulation
 * run `get_diversity` for the metapopulation


2. We create a subpopulation:
 * specify the history of transmission parameters including the number of migrant hosts per generation, $N_m$.
 * pass to `coalestr.Population()` both the history of transmission parameters and the name of the metapopulation
 * run `get_coalescent()` for the subpopulation
 * run `get_diversity` for the subpopulation
 
$F_{ST}$ is automatically calculated and returned as an attribute of the subpopulation.

In [ ]:
!pip install coalestr
from coalestr import cs
import matplotlib.pyplot as plt

In [2]:
# define the metapopulation

metapop_history = [[100000, 3000, 10, 1, 0]]
metapop = cs.Population(metapop_history)
metapop.get_coalescent()
metapop.get_diversity()

Observation time.    Events captured.   Mean coalescence time
                      beho      wiho        beho     wiho
        0              99.7      99.7     16867.3  15476.3
Observation time.    SNP heterozygosity.   Haplotype homozygosity at 27.0 kb locus
                      beho       wiho           beho       wiho
        0           3.71e-04   3.40e-04       4.41e-03   8.47e-02


In [3]:
# define the subpopulation

Nm_list = [0, 0.1, 1, 10, 30]

duration = 1000
Nh = 30
Q = 10
X = 0.5

for Nm in Nm_list:
    subpop_history = [[duration, Nh, Q, X, Nm]]

    subpop = cs.Population(subpop_history, metapopulation = metapop)
    subpop.get_coalescent(show = False)
    subpop.get_diversity(show = False)
    beho_snp_het = subpop.diversity[0,1]
    beho_hap_hom = subpop.diversity[0,3]
    fst = subpop.diversity[0,6]
    print("For Nm = {0:4.2f}: SNP het = {1:.2e}, hap hom = {2:4.2f}, Fst = {3:4.2f}".format(
        Nm,
        beho_snp_het,
        beho_hap_hom,
        fst))

For Nm = 0.00: SNP het = 2.42e-06, hap hom = 0.41, Fst = 0.99
For Nm = 0.10: SNP het = 1.54e-04, hap hom = 0.32, Fst = 0.59
For Nm = 1.00: SNP het = 3.26e-04, hap hom = 0.10, Fst = 0.12
For Nm = 10.00: SNP het = 3.67e-04, hap hom = 0.01, Fst = 0.01
For Nm = 30.00: SNP het = 3.70e-04, hap hom = 0.01, Fst = 0.00


In [ ]:
# define a set of subpopulations with different values of Nh

N_list = [30, 100, 300]
Q = 3
chi = 0.5
duration = 10000

iterations = 28

fig, ax = plt.subplots(figsize=(7,5))

for i in range(len(N_list)):
    
    Nh = N_list[i]
    M_list = []
    Fst = []
    
    for j in range(iterations):
    
        M = 1.5 ** (-j)
        Nm = Nh * M
        subpop_history = [[duration, Nh, Q, chi, Nm]]
    
        subpop = cs.Population(subpop_history, metapopulation = metapop)
        subpop.get_coalescent(show = False)
        subpop.get_diversity(show = False)
    
        M_list.append(M)
        Fst.append(subpop.diversity[0, 6])
        
    ax.plot(M_list, Fst, linewidth = 2, label = 'Nh {0:.0f}'.format(Nh))
        
ax.legend(title = 'chi {0:.2f}, Q {1:.0f}'.format(chi, Q), frameon=True, fontsize=12)
ax.set_xlabel("M", fontsize=12)
ax.set_ylabel("Fst", fontsize=12)
ax.set_xscale('log',base=10)
ax.grid(visible=True, which='both', color='0.65', linestyle='-')

plt.show()